In [28]:
## Packages need for data pre-process
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

from scipy import sparse
from collections import Counter

import nltk
from numpy import savetxt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

import itertools

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# For evaluztion (NMI)
from sklearn.metrics.cluster import normalized_mutual_info_score

# Running time
from timeit import default_timer as timer

In [2]:
# Import Dataset
df = pd.read_csv('Twitter_mani.csv')
del df['Unnamed: 0']

In [3]:
# Convert to lowercase and convert to list
data = df.text.str.lower().values.tolist()

# Remove Emails
data = [re.sub('@', '', sent) for sent in data]

# Remove hashtages
data = [re.sub('#', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove url
data = [re.sub(r'http\S+', '', sent) for sent in data]

def preProcessingFcn(tweet, 
    removeNumbers=True, removePunctuation=True):
    
    tweet = re.sub(r"\\n", " ", tweet)
    tweet = re.sub(r"&amp", " ", tweet)

    if removeNumbers==True:
        tweet=  ''.join(i for i in tweet if not i.isdigit())
    if removePunctuation==True:
        tweet = re.sub(r"[,.;@#?!&$]+\ *", " ", tweet)
  
    return tweet

data = [preProcessingFcn(tweet) for tweet in data]

In [4]:
# simple_preprocess() tokenies the text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:3])
print(len(data_words))

[[], ['leave', 'it', 'all', 'on', 'the', 'field', 'umichfootball', 'best', 'rivalry', 'in', 'college', 'football', 'goblue', 'beatosu'], ['there', 'no', 'time', 'to', 'look', 'backwards', 'only', 'ahead', 'hyped', 'to', 'watch', 'the', 'umichfootball', 'squad', 'ball', 'out', 'goblue', 'beatosu']]
12780


In [5]:
# Remove Stop Words
stop_words = stopwords.words('english')

# Define functions for stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data_words_unigrams = remove_stopwords(data_words)

# Stemming

In [6]:
data = []
for i in data_words_unigrams:
    tweet = ' '.join(i)
    data.append(tweet)

In [7]:
def stemming(tweet, stem=True):
    ps = PorterStemmer()

    if stem==True:
        tweet = ' '.join([ps.stem(word) for word in tweet.split()])
    return tweet

data_stemming = [stemming(tweet) for tweet in data]

In [8]:
data_stemming1 = []
for i in data_stemming:
    alist = i.split()
    data_stemming1.append(alist)
    
data_stemming = data_stemming1

In [9]:
# Count unique words
merged = list(itertools.chain.from_iterable(data_stemming))
print(len(merged))
print(len(set(merged)))

# Identify words that appears at least 20 times
c = Counter(merged)
a = list(Counter({k: c for k, c in c.items() if c >= 20}).keys())

# Select words that appears at least 20 times
for i,value in enumerate(data_stemming):
    data_stemming[i] = [i for i in value if i in a] 
    
# Check
merged = list(itertools.chain.from_iterable(data_stemming))
print(len(set(merged)))

# Get the index of the doc that are deleted
empty_idx = []

for i, value in enumerate(data_stemming):
    if any(value) == False:
        empty_idx.append(i)
len(empty_idx)

# Delete empty elements
data_stemming2 = list(filter(None, data_stemming))
print(len(data_stemming2))

141434
10241
1321
12439


In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_stemming2)

# Create Corpus
texts = data_stemming2

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# LDA number of clusters = 4

In [11]:
lda_model =  gensim.models.ldamodel.LdaModel(passes=10,corpus=corpus,id2word=id2word,num_topics = 4, random_state = 88)
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.022*"storm" + 0.019*"week" + 0.017*"tropic" + 0.017*"across" + 0.014*"thunderstorm" + 0.013*"flood" + 0.012*"heat" + 0.012*"weekend" + 0.011*"state" + 0.011*"weather"


Topic: 1 
Words: 0.027*"health" + 0.021*"care" + 0.018*"plan" + 0.018*"ppfa" + 0.017*"peopl" + 0.012*"parenthood" + 0.012*"get" + 0.012*"need" + 0.011*"abort" + 0.011*"women"


Topic: 2 
Words: 0.055*"goblu" + 0.031*"month" + 0.022*"michigan" + 0.016*"one" + 0.016*"year" + 0.016*"first" + 0.013*"night" + 0.011*"game" + 0.011*"day" + 0.011*"meteor"


Topic: 3 
Words: 0.082*"vegan" + 0.046*"wave" + 0.028*"recip" + 0.023*"summer" + 0.021*"made" + 0.015*"free" + 0.014*"bake" + 0.013*"make" + 0.013*"easi" + 0.012*"tofu"




# Collect true labels

In [22]:
df_new = pd.DataFrame(df.drop(df.index[empty_idx]))
df_new = df_new.reset_index(drop=True)
df_new["tokenize"] = data_stemming2

True_Label = list(df_new["user_id_new"])
len(True_Label)

12439

In [16]:
#Get the topic distribution for the given document, #doc in bow format
df_doc_topic = pd.DataFrame(columns = ['Doc_ID', 'Dominant_Topic', 'Prob_Per_Topic', 'Keywords', 'Doc_Text'])
for i in range(len(corpus)):
  a = lda_model.get_document_topics(corpus[i])
  a.sort(key = lambda x: x[1], reverse=True) #highest prob topic first
  topic_wp = lda_model.show_topic(a[0][0]) #show top 10 words byb default
  keywords = [w for (w,p) in topic_wp]
  row = [i, a[0][0], a[0][1], keywords, df_new['text'][i]]
  df_doc_topic.loc[i] = row

# Evaluation

In [24]:
# NMI
LDA_NMI = []
for i in range(0,18):
    num_cluster = i+3
    lda_model = gensim.models.ldamodel.LdaModel(passes=10,corpus=corpus,id2word=id2word,num_topics = num_cluster, random_state = 88)
    df_doc_topic = pd.DataFrame(columns = ['Dominant_Topic'])
    for i in range(len(corpus)):
        a = lda_model.get_document_topics(corpus[i])
        a.sort(key = lambda x: x[1], reverse=True) #highest prob topic first
        row = [a[0][0]]
        df_doc_topic.loc[i] = row
    pred_LDA = list(df_doc_topic["Dominant_Topic"])
    NMI = normalized_mutual_info_score(True_Label, pred_LDA)
    LDA_NMI.append(NMI)

LDA_NMI

[0.7399748575610817,
 0.8302899343978929,
 0.7733913510880616,
 0.6779834913561048,
 0.5783425578979882,
 0.4803218768670751,
 0.5389942479563443,
 0.5201222066956117,
 0.4759204956956093,
 0.5146658441288589,
 0.5308044079310624,
 0.5621249884131062,
 0.4650568948561562,
 0.45314799469794576,
 0.30815857482653414,
 0.2587460707810341,
 0.21285696651342484,
 0.22652266493368278]

In [26]:
# Purity
Correct_target = pd.DataFrame(df[["screen_name", "user_id_new"]])
Correct_target = Correct_target.drop(Correct_target.index[empty_idx]).reset_index(drop=True)
df_doc_topic = df_doc_topic.reset_index(drop=True)

LDA_purity = []
for i in range(0,18):
    num_cluster = i+3
    lda_model = gensim.models.ldamodel.LdaModel(passes=10,corpus=corpus,id2word=id2word,num_topics = num_cluster, random_state = 88)
    df_doc_topic = pd.DataFrame(columns = ['Dominant_Topic'])
    for i in range(len(corpus)):
        a = lda_model.get_document_topics(corpus[i])
        a.sort(key = lambda x: x[1], reverse=True) #highest prob topic first
        row = [a[0][0]]
        df_doc_topic.loc[i] = row
    df_compare = pd.concat([df_doc_topic, Correct_target.reindex(df_doc_topic.index)], axis=1)
    nominator = df_compare.groupby(["Dominant_Topic", "user_id_new"], as_index=False)['screen_name'].count().sort_values('screen_name', ascending=False).drop_duplicates('Dominant_Topic')["screen_name"].sum()
    purity = nominator/len(True_Label)
    LDA_purity.append(purity)

In [27]:
LDA_purity 

[0.7435485167617976,
 0.9490312726103385,
 0.9314253557359916,
 0.8899429214567087,
 0.8379290939786156,
 0.8288447624407107,
 0.8500683334673205,
 0.8453251869121312,
 0.8198408232173005,
 0.8685585658011095,
 0.8773213280810355,
 0.867191896454699,
 0.8141329688881743,
 0.8153388536055953,
 0.7158131682611142,
 0.679556234423989,
 0.6367875231127904,
 0.6382345847736957]

# Running Time Comparison

In [29]:
# The running time can also be measured simultaneously with the evaluation (when computing NMI and Purity)
LDA_running = []

for i in range(0,18):
    num_cluster = i+3
    start = timer()
    lda_model = gensim.models.ldamodel.LdaModel(passes=10,corpus=corpus,id2word=id2word,num_topics = num_cluster, random_state = 88)
    end = timer()
    time = end-start
    
    LDA_running.append(time)

In [30]:
LDA_running

[24.541619300000093,
 22.577719400000205,
 20.95631869999943,
 21.261016599999493,
 21.80628969999998,
 22.98331120000057,
 21.160482700000102,
 22.066481699999713,
 21.694230300000527,
 19.81751599999916,
 18.270815500000026,
 18.087509399999362,
 17.463955399999577,
 17.076652200000353,
 17.320095299999593,
 17.406181000000288,
 17.385939900000267,
 16.72735289999946]